In [8]:
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text  # Needed for loading universal-sentence-encoder-cmlm/multilingual-preprocess
import numpy as np

# Some texts of different lengths.
english_sentences = ["dog", "Puppies are nice.", "I enjoy taking long walks along the beach with my dog."]
italian_sentences = ["cane", "I cuccioli sono carini.", "Mi piace fare lunghe passeggiate lungo la spiaggia con il mio cane."]
japanese_sentences = ["犬", "子犬はいいです", "私は犬と一緒にビーチを散歩するのが好きです"]

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

# Compute embeddings.
en_result = embed(english_sentences)
it_result = embed(italian_sentences)
ja_result = embed(japanese_sentences)


sentence_encoding_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3", trainable=False, input_shape = [], dtype = tf.string, name = 'U.S.E')

inputs = tf.keras.layers.Input(shape = (), dtype = 'string',name = 'input_layer')
outputs = sentence_encoding_layer(inputs)

model = tf.keras.Model(inputs, outputs,name = 'Transfer_learning_USE')
model.summary()

model.predict(english_sentences)

Model: "Transfer_learning_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None,)]                 0         
                                                                 
 U.S.E (KerasLayer)          (None, 512)               68927232  
                                                                 
Total params: 68,927,232
Trainable params: 0
Non-trainable params: 68,927,232
_________________________________________________________________
1/1 [==============================] - 1s 518ms/step


array([[ 0.1009224 , -0.03814157,  0.01989768, ...,  0.06737721,
         0.01837963,  0.05331527],
       [ 0.03645061, -0.03182309,  0.06664317, ...,  0.03586333,
        -0.00165519,  0.0440184 ],
       [-0.00550115,  0.00597672, -0.01482243, ..., -0.0397649 ,
         0.03114326, -0.03666288]], dtype=float32)

: 

In [8]:
import tensorflow as tf
tf.autograph.experimental.do_not_convert
from tensorflow import keras
import tensorflow_hub as hub
import pandas as pd

model_path = "https://tfhub.dev/google/nnlm-es-dim128/2"
model_path = "https://tfhub.dev/google/nnlm-en-dim128/2"

embed = hub.load(model_path)

DATA = pd.read_pickle('data/RestaurantDataset/417c135d6d69672cd3dccf9f3d7bc369/ALL_DATA')

In [13]:
embeddings = embed(DATA.text.values)


In [16]:
embeddings

<tf.Tensor: shape=(40005, 128), dtype=float32, numpy=
array([[-0.3264753 ,  0.15007608,  0.6889177 , ..., -0.0234348 ,
         0.635548  , -0.02742715],
       [-0.49373803, -0.08537051,  0.7375126 , ...,  0.24789114,
         0.8523048 , -0.30055186],
       [-0.15956353,  0.01727004,  0.27146158, ...,  0.11012498,
         0.34013855, -0.15906505],
       ...,
       [-0.28688088,  0.01921381,  0.31409433, ...,  0.19149621,
         0.39234763, -0.05848747],
       [-0.23303813,  0.0361413 ,  0.48664016, ...,  0.05703034,
         0.36939403, -0.00979391],
       [-0.20724511,  0.04377092,  0.25718608, ...,  0.07967458,
         0.37220412, -0.05905916]], dtype=float32)>

In [6]:
hub_layer = hub.KerasLayer(model_path, input_shape=[], dtype=tf.string)
model = keras.Sequential()
model.add(hub_layer)
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()
model.predict(["Hola", "asdasdas das das da sd as das d"])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 128)               124642688 
                                                                 
 dense_6 (Dense)             (None, 16)                2064      
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 124,644,769
Trainable params: 2,081
Non-trainable params: 124,642,688
_________________________________________________________________
1/1 [==============================] - 0s 44ms/step


array([[0.45997316],
       [0.45926654]], dtype=float32)

In [4]:
import pandas as pd
import gzip, json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('/media/nas/datasets/amazon/automotive/Automotive_5.json.gz')

In [16]:
columns_name_dict = {"reviewerID":"userID", "overall": "rating", "asin":"restaurantId", "reviewText":"text", "summary":"title"}

df.rename(columns=columns_name_dict).sort_values("unixReviewTime").reset_index(drop=True)

,rating,verified,reviewTime,userID,restaurantId,style,reviewerName,text,title,unixReviewTime,vote,image
0,5.0,True,"09 14, 2000",A3TB9HNQR54B5V,B0000225EL,NaN,John,This is a great collection of connectors and a...,A great collection of accessories,968889600,12,NaN
1,1.0,False,"05 23, 2001",AY7CGSWDQ3SRL,1563923815,{'Format:': ' Paperback'},John D,I bought this Haynes book thinking that it wou...,Disappointed and Depressed,990576000,58,NaN
2,5.0,True,"04 23, 2002",A35UIYSKD6PVEG,B00NBC4IAC,{'Size:': ' 4-FY-parent9/6'},Wandering Soul,This is a nice lamp shade that fit our lamps q...,A nice shade to match the crib set.,1019520000,2,NaN
3,1.0,False,"07 22, 2002",A39XIRZI48YGZL,B00004Y774,NaN,Not Happy,Porter-Cable makes bad accesories for their po...,Another piece of [stuff] from Porter-Cable,1027296000,13,NaN
4,5.0,False,"09 24, 2002",AERYJMGYQEJZP,B00V82FB58,{'Color:': ' 1pack27'},GT,This product has differently colored sides to ...,Useful from day one.,1032825600,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1711514,2.0,False,"10 2, 2018",A2R82FNIK83TD4,B01HEHOAS4,"{'Size:': ' Large', 'Color:': ' Matte Black'}",jeff,visibility is good. noise reduction is ok........,decent for price. two stars for being useless ...,1538438400,NaN,NaN
1711515,5.0,True,"10 2, 2018",A33YDWTNM70GRG,B01H9XALY0,NaN,mickey53usa,Built an electronics project and needed a smal...,1 Amp - a little hard to find,1538438400,NaN,NaN
1711516,5.0,True,"10 2, 2018",A3LA146D5ZTSWM,B01HFMNYIU,NaN,Lisa Mackulin,The nerf bars were shipped the next day. they ...,great product,1538438400,NaN,NaN
1711517,4.0,True,"10 2, 2018",A2OY03FVH4OYKX,B01HC3QIGM,"{'Size:': ' 1 Pack', 'Color:': ' Green'}",Chako,So far so good .after much motocross.,As described.,1538438400,NaN,NaN


In [11]:
import tensorflow as tf
import numpy as np

one_weight = 1
zero_weight = 1

y_true = np.array([0., 0., 1.])
y_pred = np.array([.5, .3, .9])
b_ce = tf.keras.losses.binary_crossentropy(y_true, y_pred)

weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
weighted_b_ce = weight_vector * b_ce
loss = tf.math.reduce_mean(weighted_b_ce)

print(b_ce.numpy(), loss.numpy())

0.3850607287294289 0.3850607287294289


In [37]:
# Calculate the binary crossentropy

zero_weight = 1
one_weight = 1

epsilon = tf.keras.backend.epsilon()
y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)

# Compute cross entropy from probabilities.
bce = y_true * tf.math.log(y_pred + epsilon)
bce += (1 - y_true) * tf.math.log(1 - y_pred + epsilon)
bce = -bce

weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
weighted_bce = weight_vector * bce

print(tf.reduce_mean(bce), tf.reduce_mean(weighted_bce))

tf.Tensor(0.3850607287294289, shape=(), dtype=float64) tf.Tensor(0.7350013226099508, shape=(), dtype=float64)


In [1]:
from src.Common import get_pickle, to_pickle
import pandas as pd
import numpy as np
import sys

file_path = "data/RSTVALdataset/007570946a099bb7d79c50e21278e97e/"
file_name = "TRAIN_DEV_SEQ"

TRAIN_DEV = get_pickle(file_path, file_name)

In [5]:
TRAIN_DEV.shape

(1731660, 232)

In [ ]:
NTD = TRAIN_DEV.drop(columns=["text_base", "title_base", "text", "title", "bow", "city", "date", "language", "url", "seq"])
SQ = np.row_stack(TRAIN_DEV.seq.values).astype(np.int32)



In [ ]:
to_pickle(file_path, "TRAIN_DEV_SEQ", SQ)
